In [28]:
import io
from collections import defaultdict
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import get_dataset_dir
k=4
n=5
user="7"

data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True,
               'min_support': n
               }
algo = KNNWithMeans(k=k, min_k=k, sim_options=sim_options, verbose=True)
algo.fit(trainset)

testset = trainset.build_anti_testset()
predictions = algo.test(filter(lambda x: x[0] == user, testset))

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [29]:
def get_top_n(predictions, n):
    # First map the predictions to user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

def read_item_names():
    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = (line[1], line[2])
    return rid_to_name


In [30]:
# Read the mappings raw id <-> movie name
rid_to_name = read_item_names()
top_n = get_top_n(predictions, n)

# Print the recommended items
print("user: ", user)
for iid in top_n[user]:
    print('{:^5} {:<65} {:^5}'.format(iid[0], str(rid_to_name[str(iid[0])]), round(iid[1], 3)))

user:  7
 328  ('Conspiracy Theory (1997)', '08-Aug-1997')                         5  
1367  ('Faust (1994)', '01-Jan-1994')                                     5  
1512  ('World of Apu, The (Apur Sansar) (1959)', '05-Apr-1996')         4.968
1449  ('Pather Panchali (1955)', '22-Mar-1996')                         4.968
 315  ('Apt Pupil (1998)', '23-Oct-1998')                               4.949


In [31]:
res = []
for iid in top_n[user]:
    res.append('{} {} {}'.format(iid[0], str(rid_to_name[str(iid[0])]), round(iid[1], 3)))

result = {
    'user': user,
    'recommended items': res,
}

In [32]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
import pandas as pd
import requests
import re

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

In [33]:
def Sparql(sparql, film):
    
    queryString = """
    SELECT ?film ?screenwriterLabel ?awardsLabel
    WHERE {
      ?film wdt:P31 wd:Q11424 . # искомое является фильмом
      ?film wdt:P58 ?screenwriter . # находим сценаристов
      optional { ?screenwriter wdt:P166 ?awards . } # и полученные ими награды, если они есть
      FILTER(?film = wd:"""+film+""") # название фильма
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    results_df = pd.io.json.json_normalize(results['results']['bindings'])
    return results_df

In [37]:
# возможно это все не нужно, но
# WikiData не находит "World of Apu, The (Apur Sansar)"
# поэтому нужно оставить какое то одно название, но я сделала немного по-другому
# я разделила их на два, сначала ищу по первому названию, затем (если он не находит результатов), то по второму
# а то кто их знает, этих WikiData
print("Cписок наград, которые имеет сценарист рекомендованного фильма")

# просматриваем каждый фильм в списке рекомндаций
for iid in top_n[user]:
    film = rid_to_name[str(iid[0])][0]
    film = re.sub('\([0-9]{4}\)', "", film) # убираем год
    name1, name2 = '', ''
    #если есть дополнительное название, которое идет в скобках, мы его находим
    match = re.search('\([ a-zA-Z0-9]*\)', film)
    if match:
        # разделяем названия
        name1 = re.sub('\('+match[0]+'\)', '', film)
        name2 = re.sub('[()]', '', match[0])
    else:
        name1 = film
    
    #пробуем искать по первому названию, если все ок, то идем дальше, если нет, то ищем по второму названию
    try:
        params = {'action' : 'wbsearchentities','format' : 'json','language' : 'en','search': name1}
        res = requests.get(API_ENDPOINT, params = params)
    except:
        params = {'action' : 'wbsearchentities','format' : 'json','language' : 'en','search': name2}
        res = requests.get(API_ENDPOINT, params = params)
    print(film,":")
    #проверяем нашлелся ли фильм
    if res.json()['search']:
        results_df = Sparql(sparql, res.json()['search'][0]['id'])
        #проверяем нашлись ли сценаристы с наградами
        if len(results_df.columns) <= 0:
            print("Нет результатов\n")
        else:
            display(HTML(results_df[['film.value', 'screenwriterLabel.value', 'awardsLabel.value']].to_html()))
    else:
        print("Фильм не найден\n")    

Cписок наград, которые имеет сценарист рекомендованного фильма
Conspiracy Theory  :


HTTPError: HTTP Error 403: Forbidden

In [ ]:
import json
json_file = open("output2.json",'w')
json_file.write(json.dumps(result))
json_file.close()